In [1]:
import os
import sys
import git
import pathlib

import random
import time


import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.constants import model_seeds, error_seeds
from libs import utils
from libs.fitnessfns import ff_mnist32_cnn_ERR, eval_mnist32_cnn_ERR
from libs.ga.ga_utils import extract_best_worst_chromosomes

In [2]:
# set seed
this_seed = 84376
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        

In [4]:
# Prepare dataset
# Use only test images
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
test_images = test_images.astype(np.float32) / 255.0  
test_images = np.expand_dims(test_images, axis=-1)
# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
test_images = tf.image.resize(test_images, [image_x_size, image_y_size])

In [5]:
# Load test set
# Testing with only im_num images
im_num = 128*40#128*40=5120 #[128*78 = 9984]
test_set = (test_images[im_num:], 
            test_labels[im_num:])

In [6]:
error_profile_tag_list = []
ERR_PARAM_list = [1]#[-1,1]
MAX_FAULT_PROB_LIST = [1E-3]
error_layer_list = ["c0"]

for MAX_FAULT_PROB in MAX_FAULT_PROB_LIST:
    for seed in error_seeds[0:3]:
        err_tag = "LIM_" + f"{int(MAX_FAULT_PROB*1E3):02d}" + "-" +str(seed)
        error_profile_tag_list.append(err_tag)

In [7]:
def eval_error_injected_with_shuffle(model, 
                                      layer_error_profiles, 
                                      ERR_PARAM,
                                      shuffle_orders,
                                      test_set):
    # intialize to None
    error_profile_c0=None
    error_profile_h0=None
    error_profile_h1=None
    error_profile_h2=None
    error_profile_op=None

    error_profile_c0 = layer_error_profiles["c0"]
    error_profile_h0 = layer_error_profiles["h0"]
    error_profile_h1 = layer_error_profiles["h1"]
    error_profile_h2 = layer_error_profiles["h2"]
    error_profile_op = layer_error_profiles["op"]

    
    # intialize to None
    shuffle_order_c0 = None
    shuffle_order_h0 = None
    shuffle_order_h1 = None
    shuffle_order_h2 = None
    shuffle_order_op = None
                    
    shuffle_order_c0 = shuffle_orders["c0"]
    shuffle_order_h0 = shuffle_orders["h0"]
    shuffle_order_h1 = shuffle_orders["h1"]
    shuffle_order_h2 = shuffle_orders["h2"]
    shuffle_order_op = shuffle_orders["op"]
        
    
    avg, std = eval_mnist32_cnn_ERR(model,
                                    error_profile_c0,
                                    error_profile_h0,
                                    error_profile_h1,
                                    error_profile_h2,
                                    error_profile_op,
                                    ERR_PARAM,
                                    shuffle_order_c0,
                                    shuffle_order_h0,
                                    shuffle_order_h1,
                                    shuffle_order_h2,
                                    shuffle_order_op,
                                    test_set)
        
    return avg

In [8]:
def reset_dict_values():
    dummy_dict =  {"c0":None,
             "h0":None,
             "h1":None,
             "h2":None,
             "op":None}
    return dummy_dict

In [9]:
dataset_seg_type = "all"
print("DATASET TYPE:\t", dataset_seg_type)

DATASET TYPE:	 all


In [10]:
model_type = "mnist32-cnn_1024_256_64"
results = {}
results_filename = 'evaluate_ga_genes-mnist32-cnn--LIM_01--c0--ERR_1_-1.npy'

for model_seed in model_seeds[0:2]:#model_seeds[0:2]:
    results[model_seed] = {}
    # Get model
    model_instance = model_type + "-" + str(model_seed)
    dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
    model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
    model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
    model_filename = model_instance + ".h5"
    model_file = pathlib.Path(model_folder/ model_filename)
    # Load model
    model = tf.keras.models.load_model(model_file)
    print(model_instance)
    
    # Original Accuracy
    error_profiles = reset_dict_values()
    ERR_PARAM = None
    shuffle_orders = reset_dict_values()
    test_set = test_set
    original_accuracy = ff_mnist32_cnn_ERR(model,
                                            error_profile_c0=None,
                                            error_profile_h0=None,
                                            error_profile_h1=None,
                                            error_profile_h2=None,
                                            error_profile_op=None,
                                            ERR_PARAM=None,
                                            shuffle_order_c0=None,
                                            shuffle_order_h0=None,
                                            shuffle_order_h1=None,
                                            shuffle_order_h2=None,
                                            shuffle_order_op=None,
                                            test_set=test_set,
                                            batchsize=128).numpy()
    print("Original ACC: ", original_accuracy)
    results[model_seed]["original_acc"] = original_accuracy
    
    for error_profile_tag in error_profile_tag_list:
        print(error_profile_tag)
        results[model_seed][error_profile_tag] = {}
        # Load error profile
        error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
        error_profile_filename = error_profile_tag + ".npy"
        error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
        error_profile = np.load(error_profile_file)

        for ERR_PARAM in ERR_PARAM_list:
            print("ERR_PARAM:\t", ERR_PARAM)
            results[model_seed][error_profile_tag][ERR_PARAM]={}
            for error_layer in error_layer_list:
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]={}
                # Reset error profiles and shuffle orders for all layers
                error_profiles = reset_dict_values()
                shuffle_orders = reset_dict_values()
                
                # Inject error and observe degradation
                error_profiles[error_layer] = error_profile
                eval_acc  = eval_error_injected_with_shuffle(model, 
                                                              error_profiles, 
                                                              ERR_PARAM,
                                                              shuffle_orders,
                                                              test_set)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]["faulty"] = eval_acc
                np.save(results_filename, results) 
                print("Faulty ACC: ", eval_acc)
                
                # Use the best chromosome and observe recovered value
                best, worst = extract_best_worst_chromosomes(dataset_seg_type, # (all, qmean1, qent2)
                                                                   error_layer, 
                                                                   ERR_PARAM, 
                                                                   model_instance, 
                                                                   error_profile_tag, 
                                                                   this_seed)
                # using best chromosome
                shuffle_orders[error_layer] = best
                eval_acc  = eval_error_injected_with_shuffle(model, 
                                                              error_profiles, 
                                                              ERR_PARAM,
                                                              shuffle_orders,
                                                              test_set)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]["ga_optim"] = eval_acc
                np.save(results_filename, results) 
                print("GA ACC: ", eval_acc)

mnist32-cnn_1024_256_64-1023
Original ACC:  0.993421052631579
LIM_01-2188
ERR_PARAM:	 -1
Faulty ACC:  0.8760964912280702


KeyboardInterrupt: 

In [ ]:
sdfsd

In [ ]:
experiment = "mnist32-cnn_1024_256_64-1023--LIM_01-2188"
print(experiment)
model_instance, error_profile_tag = experiment.split("--")

In [ ]:
error_param = 1 # bitflips


In [ ]:
# Prepare dataset
# Combine test and train images together into one dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)

test_images = test_images.astype(np.float32) / 255.0  
test_images = np.expand_dims(test_images, axis=-1)

# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
test_images = tf.image.resize(test_images, [image_x_size, image_y_size]) 

In [ ]:
# Get model
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

In [ ]:
# Load model
model = tf.keras.models.load_model(model_file)

In [ ]:
# Load error profile
error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
error_profile_filename = error_profile_tag + ".npy"
error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
error_profile = np.load(error_profile_file)

In [ ]:
# Batchsize for evaluation
EVAL_BATCHSIZE = 128

In [ ]:
# Load test set
# Testing with only im_num images
im_num = 128*5#128*40=5120 #[128*78 = 9984]
test_set = (test_images[im_num:], 
            test_labels[im_num:])

In [ ]:
# def extract_best_worst_chromosomes(dataset_seg_type, # (all, qmean1, qent2)
#                                    layer, 
#                                    error_param, 
#                                    model_instance, 
#                                    error_profile_tag, 
#                                    this_seed):
    
#     dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
#     # model_instance = dataset-modelarch_modelconfig-seed
#     model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, 
#                                                                                model_arch, 
#                                                                                layer_widths, 
#                                                                                seed)
#     # model_meta_type: 'mnist32-cnn'
#     error_type = "ERR_"+str(error_param)

#     error_lim, error_seed = error_profile_tag.split('-')

#     EXP_TYPE           = dataset_seg_type + "_"  + model_meta_type + "_" + layer + "_" + error_type
#     experiment_name    = model_instance   + '--' + error_profile_tag
#     ga_experiment_name = dataset_seg_type + "_"  + experiment_name + "--" + error_type + '--' + layer + '--ga_' + str(this_seed)
#     # File/Folder to load log files
#     logging_folder = pathlib.Path(PROJ_ROOT_PATH / "logging" / dataset_seg_type / model_type / model_instance / error_lim / error_profile_tag / error_type / layer )
#     logging_filename_tag = pathlib.Path(logging_folder / ga_experiment_name)

#     # ga outputs
#     ga_output_files = glob.glob(str(logging_filename_tag) + "*.log")

#     # get chromosomes strings
#     best_chromosome, worst_chromosome = get_chromosomes(ga_output_files)
    
#     return(best_chromosome, worst_chromosome)

In [ ]:
error_params = [1] # types of error to optimize for
layers = ["c0","h2"] # layers to optimize with GA optimization

In [ ]:
for layer in layers:
    best, worst = utils.extract_best_worst_chromosomes(dataset_seg_type, # (all, qmean1, qent2)
                                                   layer, 
                                                   error_param, 
                                                   model_instance, 
                                                   error_profile_tag, 
                                                   this_seed)
    chromosomes[layer] = {"best": best,
                          "worst": worst}

In [ ]:
from libs.fitnessfns import eval_mnist32_cnn_ERR

In [ ]:
# injecting in c0 layer only
error_profiles = [error_profile, None, None, None, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("Faulty ACC [Only c0]: ", eval_acc)

In [ ]:
# injecting in c0 layer only with best GA optimization
error_profiles = [error_profile, None, None, None, None]
ERR_PARAM = -1
shuffle_orders = [chromosomes["c0"]["best"], None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [Only c0 - best]: ", eval_acc)

In [ ]:
# injecting in c0 layer only with worst GA optimization
error_profiles = [error_profile, None, None, None, None]
ERR_PARAM = -1
shuffle_orders = [chromosomes["c0"]["worst"], None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [Only c0 - worst]: ", eval_acc)

In [ ]:
# injecting in fc_2 layer only
error_profiles = [None, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("Faulty ACC [Only h2]: ", eval_acc)

In [ ]:
# injecting in fc_2 layer only with best GA optimization
error_profiles = [None, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,chromosomes["h2"]["best"], None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [Only h2 - best]: ", eval_acc)

In [ ]:
# injecting in fc_2 layer only with worst GA optimization
error_profiles = [None, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,chromosomes["h2"]["worst"], None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [Only h2 - worst]: ", eval_acc)

In [ ]:
# injecting in c0 AND fc_2 layer
error_profiles = [error_profile, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("Faulty ACC [c0 AND h2]: ", eval_acc)

In [ ]:
# injecting in c0 AND fc_2 layer with best GA optimization
error_profiles = [error_profile, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [chromosomes["c0"]["best"], None, None,chromosomes["h2"]["best"], None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [c0 AND h2] -best : ", eval_acc)

In [ ]:
# injecting in c0 AND fc_2 layer with worst GA optimization
error_profiles = [error_profile, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [chromosomes["c0"]["worst"], None, None,chromosomes["h2"]["worst"], None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [c0 AND h2] -worst : ", eval_acc)